In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 6.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
     |████████████████████████████████| 636 kB 15.6 MB/s 
     |████████████████████████████████| 895 kB 45.7 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Handling multiple sequences

* 여러 sequence 처리 방법?
* 다른 길이의 여러 sequence 처리 방법?
* vocabulary index만이 모델이 잘 돌아가도록 하는 방법인가?
* sequence가 너무 긴 것도 있는가?

## Models expect a batch of inputs

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: ignored

In [7]:
input_ids

tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])

**왜 에러가 날까??**

지금은 하나의 문장만 넣었지만 원래는 많은 문장이 들어간다.

여기서, tokenizer를 sequence에 적용할 때, input id를 tensor로 변환한 것이 아니라, 그 위에 차원을 추가한 것을 볼 수 있다.

**따라서, 차원을 더 추가해야 함!**

In [4]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward>)


Batching은 여러 문장을 동시에 모델에 보내는 행위이다!

만일, 한 개의 문장이라면, 단일 sequence로 batch 작성 가능!

In [9]:
batched_ids = [ids, ids]
batched_ids # 똑같은 2개의 문장 batch!

[[1045,
  1005,
  2310,
  2042,
  3403,
  2005,
  1037,
  17662,
  12172,
  2607,
  2026,
  2878,
  2166,
  1012],
 [1045,
  1005,
  2310,
  2042,
  3403,
  2005,
  1037,
  17662,
  12172,
  2607,
  2026,
  2878,
  2166,
  1012]]

**Batching은 여러 문장을 입력할 때 모델에 적용할 수 있다!!**

하지만, 문제는 각 문장의 길이가 다를 수 있다! => 이 문제를 해결하기 위해 padding 적용!

## Padding the inputs

In [10]:
batched_ids = [
  [200, 200, 200],
  [200, 200]
]

In [11]:
padding_id = 100

batched_ids = [
  [200, 200, 200],
  [200, 200, padding_id]
]

padding token ID는 **tokenizer.pad_token_id**를 사용하면 해결 가능하다!

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [[200, 200, 200], [200, 200, tokenizer.pad_token_id]]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)
tensor([[ 1.5694, -1.3895],
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward>)


sequence2_ids와 batched_ids의 두 번째 행은 다른 logit 값을 가지고 있다!

이렇게 되는 이유는 **Trasnformer model의 주요 기능이 각 token을 상황에 맞게 하는 attention layers이기 때문이다.**

padding token은 sequence의 모든 token을 처리하기에 padding token이 고려되어 진다!

**그러므로, 동일한 결과를 얻기 위해서는 padding token이 무시되도록 해야하고 그 작업이 attention mask를 사용하여 수행한다!**

## Attention masks

attention mask는 0s, 1s로 채워진 input id tensor이다.

1s는 해당 token에 주의를 기울러야 하고 0s는 주의를 기울리지 않아야 함을 뜻한다.

In [13]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

attention_mask = [
  [1, 1, 1],
  [1, 1, 0]
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

## 이렇게 같아질 수 있음을 알 수 있다!

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward>)


✏️ Try it out! Apply the tokenization manually on the two sentences used in section 2 (“I’ve been waiting for a HuggingFace course my whole life.” and “I hate this so much!”). Pass them through the model and check that you get the same logits as in section 2. Now batch them together using the padding token, then create the proper attention mask. Check that you obtain the same results when going through the model!

In [19]:
sequence = "I’ve been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids1 = tokenizer.convert_tokens_to_ids(tokens)

sequence = "I hate this so much!"

tokens = tokenizer.tokenize(sequence)
ids2 = tokenizer.convert_tokens_to_ids(tokens)

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [ids1]
sequence2_ids = [ids2]
batched_ids = [ids1, ids2+[tokenizer.pad_token_id]*8]
attention_mask = [
  [1]*14,
  [1]*6+[0]*8
]
print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs)

tensor([[-2.5720,  2.6852]], grad_fn=<AddmmBackward>)
tensor([[ 3.1931, -2.6685]], grad_fn=<AddmmBackward>)
SequenceClassifierOutput(loss=None, logits=tensor([[-2.5720,  2.6852],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


## Longer sequences

sequence의 길이에 제한이 있고 대부분은 512 or 1024개의 토큰 sequence를 처리한다!

그래서 더 긴 sequence를 처리할 시, 요청을 중단되는데 2가지의 해결 방법이 있다!

* 지원되는 sequence 길이가 더 긴 모델 사용
* sequence를 잘라내기

최대 길이로 잘라내기!

In [23]:
sequence = sequence[:max_sequence_length]

NameError: ignored